# Environement set up details

## Installing locally (rather than colab)

Using `uv` (from https://docs.astral.sh/uv/)

```
# install uv in its own global location (using pipx)
pipx install uv
# create a virtual environment
uv venv
# activate the environment
source .venv/bin/activate
# install the Jupyter notebook packages
uv pip install ipykernel jupyter notebook
# install required packages
uv pip install networkx matplotlib pandas mazelib imageio

In [ ]:
from typing import Tuple, List, Dict, Optional, Union, Callable, Set, Any
from dataclasses import dataclass, field, asdict
from abc import ABC, abstractmethod
import time
from datetime import datetime
from pathlib import Path
import json
from collections import deque

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import networkx as nx

from mazelib import Maze
from mazelib.generate.Sidewinder import Sidewinder
from mazelib.solve.BacktrackingSolver import BacktrackingSolver

# for creating the gif of the steps
import imageio
import matplotlib.animation as animation
import os
import tempfile

## Config

In [ ]:
@dataclass
class Config:
    """Configuration parameters for maze generation and search algorithms.

    This class centralizes all configuration options for maze environments and search
    algorithms to ensure consistent parameter usage throughout the system.

    Attributes:
        maze_size (int): Grid dimensions (nxn) of the maze. Default is 5x5.
        maze_id (Optional[int]): Seed for reproducible maze generation. If None, a random seed is used.
        visualization_delay (float): Delay in seconds between search algorithm steps for visualization. Default is 0.5.
        show_exploration (bool): Whether to visualize the exploration process. Default is True.
        max_steps (Optional[int]): Maximum steps for search algorithm execution before termination.
                                  None means unlimited steps allowed.
    """
    # Maze parameters
    maze_size: int = 5  # Grid dimensions (nxn)
    maze_id: Optional[int] = None  # Seed for reproducible maze generation

    # Visualization parameters
    visualization_delay: float = 0.5  # Delay between search steps
    show_exploration: bool = True  # Whether to visualize exploration

    # Search parameters
    max_steps: Optional[int] = None  # Maximum steps for search (None for unlimited)

## MazeEnvironment

In [ ]:
class MazeEnvironment:
    """Handles maze generation, state management and visualization.

    This class encapsulates all maze-related functionality, including generating mazes,
    managing maze state, creating graph representations for search algorithms, and
    providing visualization capabilities.

    Attributes:
        config (Config): Configuration parameters for the maze.
        grid (numpy.ndarray): Binary maze representation (0=path, 1=wall).
        start (Tuple[int, int]): Starting position, typically (1,1).
        end (Tuple[int, int]): Goal position, typically at the bottom-right corner.
        optimal_path (List[Tuple[int, int]]): Shortest solution path from start to end.
        optimal_path_length (int): Length of the shortest solution path.
        graph (Dict): Graph representation of maze for search algorithms.
    """

    def __init__(self, config: Config):
        """Initialize the maze environment with given configuration.

        Args:
            config: Configuration parameters for maze generation and visualization.
        """
        self.config = config
        self.grid = None
        self.start = None
        self.end = None
        self._maze = None
        self.seed = None
        self.optimal_path = None
        self.optimal_path_length = None
        self.graph = None
        self.generate()

    def generate(self) -> None:
        """Creates new maze using Sidewinder algorithm.

        Generates a random maze based on configuration parameters, establishes
        start and end positions, calculates optimal path, and creates graph
        representation for search algorithms.
        """
        # Use config maze_id if provided, otherwise generate random seed
        self.seed = self.config.maze_id if self.config.maze_id is not None else np.random.randint(1, 1000)

        self._maze = Maze(self.seed)
        self._maze.generator = Sidewinder(self.config.maze_size, self.config.maze_size)
        self._maze.generate()
        self._maze.generate_entrances()

        self.grid = self._maze.grid

        # Set the start to the first valid cell inside grid
        self.start = (1, 1)
        # Set the end to the last valid cell inside grid
        self.end = (self.grid.shape[0]-2, self.grid.shape[1]-2)

        # After maze generation, calculate optimal path
        self._calculate_optimal_path()

        # Create graph representation for search algorithms
        self._create_graph()

    def _calculate_optimal_path(self) -> None:
        """Calculate optimal path using maze's solver.

        Uses the BacktrackingSolver to find the optimal solution path from
        start to end. Sets optimal_path and optimal_path_length attributes.
        """
        # Set up solver
        self._maze.solver = BacktrackingSolver()
        self._maze.start = self.start
        self._maze.end = self.end

        # Solve
        self._maze.solve()

        if self._maze.solutions:
            self.optimal_path = self._maze.solutions[0]  # Store first solution
            self.optimal_path_length = len(self.optimal_path) + 1
        else:
            # Handle case where no solution is found
            self.optimal_path = None
            self.optimal_path_length = None

    def _create_graph(self) -> None:
        """Creates graph representation for search algorithms.

        Transforms the grid-based maze into an adjacency list graph representation
        where each non-wall cell is a node, and edges connect to adjacent non-wall cells.
        """
        self.graph = {}
        rows, cols = self.grid.shape

        # Define possible moves: up, right, down, left
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]

        for r in range(rows):
            for c in range(cols):
                # Skip walls
                if self.grid[r][c] == 1:
                    continue

                node = (r, c)
                neighbors = []

                # Check all four directions
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    # Check if the neighbor is valid
                    if self.is_valid_move((nr, nc)):
                        neighbors.append((nr, nc))

                self.graph[node] = neighbors

    def get_minimum_steps(self) -> Optional[int]:
        """Returns the length of optimal path if it exists.

        Returns:
            The number of steps in the optimal path from start to end,
            or None if no path exists.
        """
        return self.optimal_path_length

    def is_valid_move(self, state: Tuple[int, int]) -> bool:
        """Checks if a move to the given state is legal.

        Args:
            state: Coordinates (row, col) to check for validity.

        Returns:
            True if the move is valid (within bounds and not a wall),
            False otherwise.
        """
        row, col = state
        # check if the move goes outside of the grid or into a wall (1)
        if (row < 0 or row >= self.grid.shape[0] or
            col < 0 or col >= self.grid.shape[1] or
            self.grid[state] == 1):
            return False
        # move is valid
        return True

    def visualize(self, path: Optional[List[Tuple[int, int]]] = None,
                  visited: Optional[Set[Tuple[int, int]]] = None,
                  show_optimal: bool = False,
                  save_path: Optional[Path] = None,
                  title: Optional[str] = None) -> None:
        """Displays or saves maze visualization with optional path and visited nodes.

        Creates a color-coded visualization of the maze showing walls, paths,
        visited nodes, and solution paths.

        Args:
            path: Optional list of positions showing a solution path.
            visited: Optional set of visited positions during search.
            show_optimal: Whether to display the optimal path.
            save_path: If provided, saves figure to this path instead of displaying.
            title: Optional title for the plot.
        """
        plt.figure(figsize=(8, 8))

        # Add title showing Maze ID and minimum steps
        if title:
            plt.title(title)
        else:
            plt.title(f"Maze #{self.seed} - Min Steps: {self.get_minimum_steps()}")

        # Create a visualization grid filled with appropriate values for coloring
        rows, cols = self.grid.shape
        viz_grid = np.ones((rows, cols)) * 5  # Initialize with unvisited path value

        # Fill in walls
        viz_grid[self.grid == 1] = 0  # Walls

        # Fill in visited paths
        if visited:
            for pos in visited:
                r, c = pos
                if self.grid[r, c] == 0:  # Only if it's a path
                    viz_grid[r, c] = 1  # Visited paths

        # Fill in final path
        if path:
            for pos in path:
                if pos != self.start and pos != self.end:
                    r, c = pos
                    viz_grid[r, c] = 2  # Final path

        # Mark start and end
        viz_grid[self.start] = 3  # Start
        viz_grid[self.end] = 4    # Goal

        # Define colors: Wall, Visited, Final Path, Start, Goal, Unvisited
        cmap = plt.cm.colors.ListedColormap(['black', 'yellow', 'green', 'blue', 'purple', 'white'])
        bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
        norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

        # Plot the maze
        plt.imshow(viz_grid, cmap=cmap, norm=norm)
        plt.xticks([])
        plt.yticks([])

        # Add legend
        legend_elements = [
            plt.Rectangle((0,0), 1, 1, color='white', label='Path'),
            plt.Rectangle((0,0), 1, 1, color='yellow', label='Visited'),
            plt.Rectangle((0,0), 1, 1, color='black', label='Wall'),
            plt.Rectangle((0,0), 1, 1, color='green', label='Final Path'),
            plt.Rectangle((0,0), 1, 1, color='blue', label='Start'),
            plt.Rectangle((0,0), 1, 1, color='purple', label='Goal')
        ]
        plt.legend(handles=legend_elements, loc='upper center',
                  bbox_to_anchor=(0.5, -0.05), ncol=3)

        if save_path:
            plt.savefig(save_path, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def calculate_manhattan_distance(self, state: Tuple[int, int], goal: Tuple[int, int]) -> int:
        """Calculate Manhattan distance heuristic from state to goal."""
        return abs(state[0] - goal[0]) + abs(state[1] - goal[1])

    def calculate_euclidean_distance(self, state: Tuple[int, int], goal: Tuple[int, int]) -> float:
        """Calculate Euclidean distance heuristic from state to goal."""
        return ((state[0] - goal[0]) ** 2 + (state[1] - goal[1]) ** 2) ** 0.5

    def get_step_cost(self, state1: Tuple[int, int], state2: Tuple[int, int]) -> int:
        """Calculate cost of moving from state1 to state2."""
        # For uniform cost in grid-based maze, return 1
        return 1

## SearchResult

In [ ]:
@dataclass
class SearchResult:
    """Enhanced container for search algorithm results with educational metrics.

    This class stores and analyzes the results of search algorithm execution, providing
    not only basic path information but also educational metrics and insights about
    algorithm performance. It includes capabilities for generating reports and
    visualizations of search results.

    Attributes:
        path (List[Tuple[int, int]]): Solution path from start to goal, if found.
        visited (List[Tuple[int, int]]): List of nodes visited during search, in order.
        success (bool): Whether the search found a valid path to the goal.
        steps (int): Number of algorithm steps executed during search.
        execution_time (float): Time taken for the search execution in seconds.
        exploration_history (List): History of algorithm state for visualization/analysis.
        node_discovery (Dict): Maps each node to the step when it was first discovered.
        node_expansion (Dict): Maps each node to the step when it was expanded.
    """
    path: Optional[List[Tuple[int, int]]] = None  # Solution path from start to goal
    visited: List[Tuple[int, int]] = field(default_factory=list)  # List of visited nodes
    success: bool = False  # Whether the search found a path
    steps: int = 0  # Number of algorithm steps executed
    execution_time: float = 0.0  # Time taken for the search
    exploration_history: List = field(default_factory=list)  # History of algorithm state (for visualization)

    # Educational tracking metrics
    node_discovery: Dict[Tuple[int, int], int] = field(default_factory=dict)  # When each node was discovered
    node_expansion: Dict[Tuple[int, int], int] = field(default_factory=dict)  # When each node was expanded

    def __str__(self) -> str:
        """Enhanced string representation of results with educational metrics.

        Returns:
            A formatted string with key performance metrics and educational information.
        """
        if self.success:
            return self._format_success_output()
        else:
            return self._format_failure_output()

    def _format_success_output(self) -> str:
        """Format output for successful search with educational information.

        Returns:
            A formatted string with performance metrics for successful searches.
        """
        avg_branching = self._calculate_avg_branching_factor()

        output = [
            f"✅ Search succeeded in {self.steps} steps ({self.execution_time:.3f}s)",
            f"📏 Path length: {len(self.path)} nodes",
            f"🔍 Visited nodes: {len(self.visited)} nodes ({(len(self.visited) / self.steps):.2f} nodes/step)",
            f"⚙️ Efficiency: {(len(self.path) / len(self.visited) * 100):.1f}% (path nodes / visited nodes)",
            f"🌲 Average branching factor: {avg_branching:.2f} neighbors/node",
            f"⏱️ Average time per step: {(self.execution_time / self.steps * 1000):.2f} ms"
        ]

        return "\n".join(output)

    def _format_failure_output(self) -> str:
        """Format output for failed search with educational information.

        Returns:
            A formatted string with performance metrics and possible failure reasons.
        """
        avg_branching = self._calculate_avg_branching_factor()

        output = [
            f"❌ Search failed after {self.steps} steps ({self.execution_time:.3f}s)",
            f"🔍 Visited nodes: {len(self.visited)} nodes ({(len(self.visited) / self.steps):.2f} nodes/step)",
            f"🌲 Average branching factor: {avg_branching:.2f} neighbors/node",
            f"⏱️ Average time per step: {(self.execution_time / self.steps * 1000):.2f} ms",
            f"💡 Possible reasons for failure:",
            f"   - No valid path exists between start and goal",
            f"   - Search exceeded maximum step limit ({self.steps} steps)",
            f"   - Maze structure prevents reaching the goal"
        ]

        return "\n".join(output)

    def _calculate_avg_branching_factor(self) -> float:
        """Calculate the average branching factor during search.

        Returns:
            The average number of new nodes discovered per expanded node,
            which approximates the branching factor of the search space.
        """
        if not self.node_expansion or len(self.node_expansion) <= 1:
            return 0.0

        # We can determine this from the ratio of discovered nodes to expanded nodes
        # Excluding the start node which doesn't have a parent
        discovered_count = len(self.node_discovery) - 1  # -1 for start node
        expanded_count = len(self.node_expansion)

        return discovered_count / expanded_count if expanded_count > 0 else 0.0

    def to_dict(self) -> Dict[str, Any]:
        """Convert results to dictionary for analysis with educational metrics.

        Returns:
            Dictionary containing all metrics and performance data for analysis.
        """
        return {
            'success': self.success,
            'steps': self.steps,
            'execution_time': self.execution_time,
            'path_length': len(self.path) if self.path else None,
            'visited_nodes': len(self.visited),
            'path_to_visited_ratio': (len(self.path) / len(self.visited)
                                     if self.path and len(self.visited) > 0 else None),
            'avg_branching_factor': self._calculate_avg_branching_factor(),
            'discovery_to_expansion_ratio': (len(self.node_discovery) / len(self.node_expansion)
                                           if self.node_expansion else None),
            'nodes_per_step': len(self.visited) / self.steps if self.steps > 0 else 0
        }


## Base Classes - SearchAlgorithmBase, UniformedSearch, and InformedSearch

In [ ]:

class SearchAlgorithmBase(ABC):
    """Abstract base class for search algorithms with enhanced shared functionality."""

    def __init__(self, env: MazeEnvironment):
        """Initialize the search algorithm with an environment."""
        self.env = env
        self.config = env.config
        self.name = self.__class__.__name__

    def _reconstruct_path(self, parent, start, goal):
        """Reconstruct the solution path from parent pointers."""
        path = [goal]
        current = goal
        while current != start:
            current = parent[current]
            path.append(current)
        return path[::-1]  # Reverse to get start→goal

    def create_search_result(self, path, visited_order, success, steps, exploration_history, **kwargs):
        """Create a standardized SearchResult with support for additional metrics."""
        result = SearchResult(
            path=path,
            visited=visited_order,
            success=success,
            steps=steps,
            exploration_history=exploration_history
        )
        # Add additional metrics that vary by algorithm
        for key, value in kwargs.items():
            setattr(result, key, value)
        return result

    @abstractmethod
    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Search for a path from start to goal."""
        raise NotImplementedError("Subclasses must implement search method")

    def run(self, start: Optional[Tuple[int, int]] = None,
            goal: Optional[Tuple[int, int]] = None) -> SearchResult:
        """Run the search algorithm with timing and error handling."""
        # Use environment start/goal if not specified
        start = start if start is not None else self.env.start
        goal = goal if goal is not None else self.env.end

        start_time = time.time()
        try:
            result = self.search(start, goal)
        except Exception as e:
            print(f"Error in {self.name}: {str(e)}")
            result = SearchResult(success=False)

        result.execution_time = time.time() - start_time
        return result

    @abstractmethod
    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Visualize the search process step by step."""
        raise NotImplementedError("Subclasses must implement visualize_search method")

### Base class for Uninformed Search

In [ ]:
class UninformedSearch(SearchAlgorithmBase):
    """Base class for uninformed search algorithms (BFS, DFS).

    Uninformed search algorithms don't use domain knowledge beyond the problem definition.
    They differ primarily in their frontier data structure and expansion strategy.
    """

    @abstractmethod
    def _initialize_frontier(self, start):
        """Initialize frontier data structure with start node."""
        raise NotImplementedError("Subclasses must implement _initialize_frontier")

    @abstractmethod
    def _get_next_node(self, frontier):
        """Get the next node from frontier based on search strategy (FIFO, LIFO, etc.)."""
        raise NotImplementedError("Subclasses must implement _get_next_node")

    @abstractmethod
    def _add_to_frontier(self, frontier, node):
        """Add a node to frontier according to search strategy."""
        raise NotImplementedError("Subclasses must implement _add_to_frontier")

    @abstractmethod
    def _frontier_representation(self, frontier):
        """Return a representation of frontier suitable for visualization."""
        raise NotImplementedError("Subclasses must implement _frontier_representation")

    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Generic uninformed search implementation."""
        # Initialize data structures
        frontier = self._initialize_frontier(start)
        visited = set([start])
        visited_order = [start]
        parent = {start: None}
        exploration_history = []
        node_discovery = {start: 0}
        node_expansion = {}

        steps = 0
        max_steps = self.config.max_steps or float('inf')

        while frontier and steps < max_steps:
            steps += 1

            # Save frontier state for visualization if needed
            frontier_before = self._frontier_representation(frontier) if self.config.show_exploration else None

            # Get next node to explore (algorithm-specific)
            current_node = self._get_next_node(frontier)
            node_expansion[current_node] = steps

            # Check if goal is reached
            if current_node == goal:
                final_path = self._reconstruct_path(parent, start, goal)
                return self.create_search_result(
                    path=final_path,
                    visited_order=visited_order,
                    success=True,
                    steps=steps,
                    exploration_history=exploration_history,
                    node_discovery=node_discovery,
                    node_expansion=node_expansion
                )

            # Process neighbors
            neighbors_added = []
            for neighbor in self.env.graph.get(current_node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    visited_order.append(neighbor)
                    parent[neighbor] = current_node
                    self._add_to_frontier(frontier, neighbor)
                    node_discovery[neighbor] = steps
                    neighbors_added.append(neighbor)

            # Record exploration history
            if self.config.show_exploration:
                current_partial_path = self._reconstruct_path(parent, start, current_node) if current_node != start else [start]

                # Create step info dictionary (using algorithm-specific names)
                step_info = self._create_step_info(
                    current_node, steps, neighbors_added,
                    frontier_before, self._frontier_representation(frontier)
                )

                exploration_history.append((
                    visited.copy(),
                    self._frontier_representation(frontier),
                    current_partial_path,
                    step_info
                ))

        # No path found
        return self.create_search_result(
            path=None,
            visited_order=visited_order,
            success=False,
            steps=steps,
            exploration_history=exploration_history,
            node_discovery=node_discovery,
            node_expansion=node_expansion
        )

    def _create_step_info(self, current_node, steps, neighbors_added, frontier_before, frontier_after):
        """Create step info dictionary for visualization."""
        # Subclasses can override to add algorithm-specific fields
        return {
            "step": steps,
            "expanded_node": current_node,
            "neighbors_added": neighbors_added,
            "frontier_before": frontier_before,
            "frontier_after": frontier_after
        }

    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Visualize the search process step by step."""
        if not result.exploration_history:
            print(f"No exploration history available for {self.name}")
            return

        delay = delay if delay is not None else self.config.visualization_delay

        for i, state in enumerate(result.exploration_history):
            # Clear previous output
            clear_output(wait=True)

            # Unpack exploration history
            visited, frontier, current_path, step_info = state

            # Display title based on search progress
            if i == len(result.exploration_history) - 1 and result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
                current_path = result.path
            else:
                title = f"{self.name} - Step {i+1}/{len(result.exploration_history)}"

            # Display the maze visualization
            self.env.visualize(path=current_path, visited=visited, title=title)

            # Format node coordinates for display
            def format_node(node):
                return f"({node[0]},{node[1]})"

            # Display educational commentary
            print(f"Step {step_info['step']}: Expanding node {format_node(step_info['expanded_node'])}")

            # Display frontier information (generic terms)
            print(f"Frontier size: {len(frontier)}")
            print(f"Visited nodes: {len(visited)}")
            print(f"Current path length: {len(current_path) if current_path else 0}")

            plt.pause(delay)

        # Show final state if exploration wasn't saved
        if not self.config.show_exploration:
            clear_output(wait=True)
            if result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
            else:
                title = f"{self.name} - No path found. ({result.steps} steps, {result.execution_time:.3f}s)"

            self.env.visualize(path=result.path, visited=set(result.visited), title=title)

        print(result)

### Base Class for Informed Search

In [ ]:
class InformedSearch(SearchAlgorithmBase):
    """Base class for informed search algorithms (Greedy Best-First, A*).

    Informed search algorithms use domain knowledge (heuristics) to guide the search.
    """

    def _initialize_metrics(self, start, goal):
        """Initialize common metrics for informed search algorithms."""
        h_start = self.env.calculate_manhattan_distance(start, goal)

        return {
            "node_discovery": {start: 0},
            "node_expansion": {},
            "node_h_value": {start: h_start},
            "node_g_value": {start: 0},
            "node_f_value": {start: self._calculate_f(0, h_start)}
        }

    @abstractmethod
    def _calculate_f(self, g, h):
        """Calculate f-value based on g and h (varies by algorithm)."""
        pass

    @abstractmethod
    def _should_update_node(self, neighbor, tentative_g, g_value, frontier_dict):
        """Determine if a node's path should be updated."""
        pass

    def _create_step_info(self, current_node, steps, neighbors_added, frontier_before,
                          frontier_after, metrics):
        """Create step info dictionary for informed search visualization."""
        return {
            "step": steps,
            "expanded_node": current_node,
            "expanded_node_f": metrics["node_f_value"][current_node],
            "expanded_node_g": metrics["node_g_value"][current_node],
            "expanded_node_h": metrics["node_h_value"][current_node],
            "neighbors_added": neighbors_added,
            "frontier_before": frontier_before,
            "frontier_after": frontier_after
        }

## BreadthFirstSearch

In [ ]:
class BreadthFirstSearch(UninformedSearch):
    """Breadth-First Search implementation."""

    def _initialize_frontier(self, start):
        """Initialize a queue with the start node."""
        return deque([start])

    def _get_next_node(self, frontier):
        """Get the next node from the queue (FIFO)."""
        return frontier.popleft()

    def _add_to_frontier(self, frontier, node):
        """Add a node to the end of the queue."""
        frontier.append(node)

    def _frontier_representation(self, frontier):
        """Convert queue to list for visualization."""
        return list(frontier)

    def _create_step_info(self, current_node, steps, neighbors_added, frontier_before, frontier_after):
        """Create step info with queue terminology."""
        info = super()._create_step_info(current_node, steps, neighbors_added, frontier_before, frontier_after)
        info["queue_before"] = frontier_before
        info["queue_after"] = frontier_after
        return info


## DepthFirstSearch

In [ ]:
class DepthFirstSearch(UninformedSearch):
    """Depth-First Search implementation."""

    def _initialize_frontier(self, start):
        """Initialize a stack with the start node."""
        return [start]

    def _get_next_node(self, frontier):
        """Get the next node from the stack (LIFO)."""
        return frontier.pop()

    def _add_to_frontier(self, frontier, node):
        """Add a node to the top of the stack."""
        frontier.append(node)

    def _frontier_representation(self, frontier):
        """Return stack as list for visualization."""
        return list(frontier)

    def _create_step_info(self, current_node, steps, neighbors_added, frontier_before, frontier_after):
        """Create step info with stack terminology."""
        info = super()._create_step_info(current_node, steps, neighbors_added, frontier_before, frontier_after)
        info["stack_before"] = frontier_before
        info["stack_after"] = frontier_after
        return info

## Greedy Best First Search

In [ ]:
class GreedyBestFirstSearch(InformedSearch):
    """Greedy Best-First Search implementation."""

    def _calculate_f(self, g, h):
        """For Greedy Best-First Search, f = h."""
        return h

    def _should_update_node(self, neighbor, tentative_g, g_value, frontier_dict):
        """Update if node is not in frontier."""
        return neighbor not in frontier_dict

    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Perform Greedy Best-First Search."""
        # Initialize metrics
        metrics = self._initialize_metrics(start, goal)

        # Initialize priority queue (using list with sorting)
        frontier = [(metrics["node_h_value"][start], start)]  # (f=h, node)
        frontier_dict = {start: metrics["node_h_value"][start]}

        # Initialize tracking variables
        visited_order = [start]
        closed_set = set()  # Nodes already expanded
        parent = {start: None}
        g_value = {start: 0}  # Still tracking g for path reconstruction
        exploration_history = []

        steps = 0
        max_steps = self.config.max_steps or float('inf')

        while frontier and steps < max_steps:
            steps += 1

            # Sort frontier by h-value (which is f for Greedy)
            frontier.sort(key=lambda x: x[0])
            frontier_before = frontier.copy() if self.config.show_exploration else None

            # Get node with lowest h-value
            h_value, current_node = frontier.pop(0)
            del frontier_dict[current_node]

            # Add to closed set
            closed_set.add(current_node)
            metrics["node_expansion"][current_node] = steps

            # Check if goal reached
            if current_node == goal:
                final_path = self._reconstruct_path(parent, start, goal)
                return self.create_search_result(
                    path=final_path,
                    visited_order=visited_order,
                    success=True,
                    steps=steps,
                    exploration_history=exploration_history,
                    **metrics
                )

            # Process neighbors
            neighbors_added = []
            for neighbor in self.env.graph.get(current_node, []):
                if neighbor in closed_set:
                    continue

                # Calculate tentative g-value (not used for expansion decisions but for tracking)
                tentative_g = g_value[current_node] + self.env.get_step_cost(current_node, neighbor)

                # Check if we should update this node
                if self._should_update_node(neighbor, tentative_g, g_value, frontier_dict):
                    # Update path info
                    parent[neighbor] = current_node
                    g_value[neighbor] = tentative_g

                    # Calculate heuristic
                    h = self.env.calculate_manhattan_distance(neighbor, goal)
                    f = self._calculate_f(tentative_g, h)  # For Greedy, f = h

                    # Update metrics
                    if neighbor not in metrics["node_discovery"]:
                        metrics["node_discovery"][neighbor] = steps
                        visited_order.append(neighbor)

                    metrics["node_h_value"][neighbor] = h
                    metrics["node_g_value"][neighbor] = tentative_g
                    metrics["node_f_value"][neighbor] = f

                    # Update frontier
                    if neighbor in frontier_dict:
                        # Remove old entry (frontier contains tuples)
                        frontier = [(f_val, n) for f_val, n in frontier if n != neighbor]

                    # Add with new values
                    frontier.append((f, neighbor))
                    frontier_dict[neighbor] = f
                    neighbors_added.append((f, neighbor))

            # Record exploration history
            if self.config.show_exploration:
                current_partial_path = self._reconstruct_path(parent, start, current_node) if current_node != start else [start]

                step_info = self._create_step_info(
                    current_node, steps, neighbors_added,
                    frontier_before, frontier.copy(),
                    metrics
                )

                exploration_history.append((
                    closed_set.copy(),
                    frontier.copy(),
                    current_partial_path,
                    step_info
                ))

        # No path found
        return self.create_search_result(
            path=None,
            visited_order=visited_order,
            success=False,
            steps=steps,
            exploration_history=exploration_history,
            **metrics
        )

    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Visualize Greedy Best-First Search with heuristic information."""
        if not result.exploration_history:
            print(f"No exploration history available for {self.name}")
            return

        delay = delay if delay is not None else self.config.visualization_delay

        for i, state in enumerate(result.exploration_history):
            # Clear previous output
            clear_output(wait=True)

            # Unpack exploration history
            closed_set, frontier, current_path, step_info = state

            # Format node coordinates for display
            def format_node(node):
                return f"({node[0]},{node[1]})"

            # Educational commentary specific to Greedy Best-First Search
            expanded_node = format_node(step_info["expanded_node"])
            commentary = [
                f"Step {step_info['step']}: Expanding node {expanded_node} with h={step_info['expanded_node_h']:.2f}",
                f"Greedy Best-First Search always expands the node with the lowest heuristic value."
            ]

            # Add information about neighbors discovered
            if step_info["neighbors_added"]:
                neighbors_text = ", ".join(
                    [f"{format_node(n)} (h={h:.2f})" for h, n in step_info["neighbors_added"]]
                )
                commentary.append(f"Discovered neighbors: {neighbors_text}")
            else:
                commentary.append(f"No new neighbors were discovered.")

            # Information about frontier
            frontier_text = ", ".join(
                [f"{format_node(n)} (h={h:.2f})" for h, n in frontier]
            )
            commentary.append(f"Frontier: [{frontier_text}]")

            # Display title based on progress
            if i == len(result.exploration_history) - 1 and result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
                current_path = result.path
            else:
                title = f"{self.name} - Step {i+1}/{len(result.exploration_history)}"

            # Display the maze visualization
            self.env.visualize(path=current_path, visited=closed_set, title=title)

            # Display educational commentary
            print("\n".join(commentary))

            # Additional stats
            print(f"\nExpanded nodes: {len(closed_set)}")
            print(f"Frontier size: {len(frontier)}")
            print(f"Current path length: {len(current_path) if current_path else 0}")

            plt.pause(delay)

        # Show final state if exploration wasn't saved
        if not self.config.show_exploration:
            clear_output(wait=True)
            if result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
            else:
                title = f"{self.name} - No path found. ({result.steps} steps, {result.execution_time:.3f}s)"

            self.env.visualize(path=result.path, visited=set(result.visited), title=title)

        print(result)

## A* Search

In [ ]:
class AStarSearch(SearchAlgorithmBase):
    """A* Search implementation with educational output.

    A* combines the cost-so-far (g-value) with a heuristic estimate (h-value)
    to guide search. It balances finding a short path with finding it quickly,
    and is guaranteed to find the optimal path if the heuristic is admissible.
    """

    def _reconstruct_path(self, parent, start, goal):
        """Reconstruct the solution path from parent pointers."""
        # Same as in previous implementations
        path = [goal]
        current = goal
        while current != start:
            current = parent[current]
            path.append(current)
        return path[::-1]

    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Perform A* Search from start to goal."""
        # Calculate initial h-value
        h_start = self.env.calculate_manhattan_distance(start, goal)

        # Priority queue (list that we'll sort)
        # Each entry is (f_value, g_value, node) - f = g + h
        frontier = [(h_start, 0, start)]  # f=h+g, g=0 for start node
        frontier_dict = {start: h_start}  # Maps node -> f_value

        # Tracking variables
        closed_set = set()  # Nodes already expanded
        visited_order = [start]
        parent = {start: None}
        g_value = {start: 0}  # Cost from start to node

        exploration_history = []
        node_discovery = {start: 0}
        node_expansion = {}
        node_h_value = {start: h_start}
        node_g_value = {start: 0}
        node_f_value = {start: h_start}

        steps = 0
        max_steps = self.config.max_steps or float('inf')

        while frontier and steps < max_steps:
            steps += 1

            # Sort by f_value (and break ties with g_value)
            frontier.sort(key=lambda x: (x[0], -x[1]))  # Sort by f, then by -g (prefer higher g if same f)
            f, g, current_node = frontier.pop(0)
            del frontier_dict[current_node]

            # Add to closed set
            closed_set.add(current_node)
            node_expansion[current_node] = steps

            # Goal check
            if current_node == goal:
                final_path = self._reconstruct_path(parent, start, goal)
                result = SearchResult(
                    path=final_path,
                    visited=visited_order,
                    success=True,
                    steps=steps,
                    exploration_history=exploration_history
                )
                # Add educational data
                result.node_discovery = node_discovery
                result.node_expansion = node_expansion
                result.node_h_value = node_h_value
                result.node_g_value = node_g_value
                result.node_f_value = node_f_value
                return result

            # Process neighbors
            neighbors_added = []
            for neighbor in self.env.graph.get(current_node, []):
                if neighbor in closed_set:
                    continue

                # Calculate new g-value
                tentative_g = g_value[current_node] + self.env.get_step_cost(current_node, neighbor)

                # If this node is new OR we found a better path to it
                if neighbor not in frontier_dict or tentative_g < g_value[neighbor]:
                    # Update tracking info
                    parent[neighbor] = current_node
                    g_value[neighbor] = tentative_g

                    # Calculate f-value
                    h = self.env.calculate_manhattan_distance(neighbor, goal)
                    f = tentative_g + h

                    # Add to educational tracking
                    if neighbor not in node_discovery:
                        node_discovery[neighbor] = steps
                        visited_order.append(neighbor)

                    node_h_value[neighbor] = h
                    node_g_value[neighbor] = tentative_g
                    node_f_value[neighbor] = f

                    # Update frontier
                    if neighbor in frontier_dict:
                        # Remove old entry
                        frontier = [(f_val, g_val, n) for f_val, g_val, n in frontier if n != neighbor]

                    frontier.append((f, tentative_g, neighbor))
                    frontier_dict[neighbor] = f
                    neighbors_added.append((f, tentative_g, neighbor))

            # Record exploration history
            if self.config.show_exploration:
                current_partial_path = self._reconstruct_path(parent, start, current_node) if current_node != start else [start]

                step_info = {
                    "step": steps,
                    "expanded_node": current_node,
                    "expanded_node_f": f,
                    "expanded_node_g": g,
                    "expanded_node_h": node_h_value[current_node],
                    "neighbors_added": neighbors_added,
                    "frontier_before": [(f_val, g_val, n) for f_val, g_val, n in frontier
                                      if n not in [node for _, _, node in neighbors_added]],
                    "frontier_after": frontier.copy()
                }

                exploration_history.append((
                    closed_set.copy(),
                    frontier.copy(),
                    current_partial_path,
                    step_info
                ))

        # No path found
        result = SearchResult(
            path=None,
            visited=visited_order,
            success=False,
            steps=steps,
            exploration_history=exploration_history
        )
        # Add educational data
        result.node_discovery = node_discovery
        result.node_expansion = node_expansion
        result.node_h_value = node_h_value
        result.node_g_value = node_g_value
        result.node_f_value = node_f_value
        return result

    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Visualize A* Search with educational information about f, g, h values."""
        if not result.exploration_history:
            print(f"No exploration history available for {self.name}")
            return

        delay = delay if delay is not None else self.config.visualization_delay

        for i, state in enumerate(result.exploration_history):
            # Clear previous output
            clear_output(wait=True)

            # Unpack exploration history
            closed_set, frontier, current_path, step_info = state

            # Format node coordinates for display
            def format_node(node):
                return f"({node[0]},{node[1]})"

            # Educational commentary specific to A* Search
            expanded_node = format_node(step_info["expanded_node"])
            f_value = step_info["expanded_node_f"]
            g_value = step_info["expanded_node_g"]
            h_value = step_info["expanded_node_h"]

            commentary = [
                f"Step {step_info['step']}: Expanding node {expanded_node}",
                f"f(n) = g(n) + h(n) = {g_value:.2f} + {h_value:.2f} = {f_value:.2f}"
            ]

            # Add information about neighbors
            if step_info["neighbors_added"]:
                neighbors_text = ", ".join(
                    [f"{format_node(n)} (f={f:.2f})" for f, g, n in step_info["neighbors_added"]]
                )
                commentary.append(f"Discovered neighbors: {neighbors_text}")
            else:
                commentary.append(f"No new neighbors were discovered")

            # Display title based on progress
            if i == len(result.exploration_history) - 1 and result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
                current_path = result.path
            else:
                title = f"{self.name} - Step {i+1}/{len(result.exploration_history)}"

            # Display the maze visualization
            self.env.visualize(path=current_path, visited=closed_set, title=title)

            # Display educational commentary
            print("\n".join(commentary))

            # Additional stats
            print(f"\nExpanded nodes: {len(closed_set)}")
            print(f"Frontier size: {len(frontier)}")
            print(f"Current path length: {len(current_path) if current_path else 0}")

            plt.pause(delay)

        # Show final state if exploration wasn't saved
        if not self.config.show_exploration:
            clear_output(wait=True)
            if result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
            else:
                title = f"{self.name} - No path found. ({result.steps} steps, {result.execution_time:.3f}s)"

            self.env.visualize(path=result.path, visited=set(result.visited), title=title)

        print(result)

## Dashboards

In [ ]:
from abc import ABC, abstractmethod
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import os
import tempfile
import imageio
from typing import List, Tuple, Dict, Set, Optional

class SearchAlgorithmDashboard(ABC):
    """Abstract base class for search algorithm educational dashboards."""

    def __init__(self, env: MazeEnvironment, result: SearchResult):
        """Initialize the educational dashboard with search results."""
        self.env = env
        self.result = result
        self.algorithm_name = self.__class__.__name__.replace("Dashboard", "")
        self.steps_data = []
        self._extract_history_data()

    @abstractmethod
    def _extract_history_data(self):
        """Process exploration history into data for visualization."""
        pass

    @abstractmethod
    def get_frontier_name(self) -> str:
        """Return name of frontier data structure (Queue, Stack, etc.)."""
        pass

    def create_maze_graph(self):
        """Create a NetworkX graph from maze data."""
        G = nx.Graph()

        # Add nodes for all valid positions
        for r in range(self.env.grid.shape[0]):
            for c in range(self.env.grid.shape[1]):
                if self.env.grid[r, c] == 0:  # If not a wall
                    G.add_node((r, c))

        # Add edges between adjacent nodes
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        for node in G.nodes():
            r, c = node
            for dr, dc in directions:
                neighbor = (r + dr, c + dc)
                if neighbor in G:  # If neighbor exists in graph
                    G.add_edge(node, neighbor)

        return G

    def _plot_maze(self, ax, step_data):
        """Plot the maze with current path and visited nodes."""
        rows, cols = self.env.grid.shape
        viz_grid = np.ones((rows, cols)) * 5  # Initialize with unvisited path value

        # Fill in walls
        viz_grid[self.env.grid == 1] = 0  # Walls

        # Fill in visited paths
        for pos in step_data['visited']:
            r, c = pos
            if self.env.grid[r, c] == 0:  # Only if it's a path
                viz_grid[r, c] = 1  # Visited paths

        # Fill in current frontier
        # Use frontier_nodes if available (for informed search), otherwise use frontier
        frontier_to_plot = step_data.get('frontier_nodes', step_data['frontier'])
        for pos in frontier_to_plot:
            r, c = pos
            if pos != self.env.start and pos != self.env.end:
                viz_grid[r, c] = 6  # Frontier nodes

        # Fill in expanded node
        if step_data['expanded_node'] != self.env.start and step_data['expanded_node'] != self.env.end:
            r, c = step_data['expanded_node']
            viz_grid[r, c] = 7  # Current expanded node

        # Fill in current path
        if step_data['current_path']:
            for pos in step_data['current_path']:
                if pos != self.env.start and pos != self.env.end:
                    r, c = pos
                    viz_grid[r, c] = 2  # Current path

        # Mark start and end
        viz_grid[self.env.start] = 3  # Start
        viz_grid[self.env.end] = 4    # Goal

        # Define colors with enhanced palette
        cmap = plt.cm.colors.ListedColormap([
            'black',     # 0: Wall
            'yellow',    # 1: Visited
            'green',     # 2: Current path
            'blue',      # 3: Start
            'purple',    # 4: Goal
            'white',     # 5: Unvisited path
            'lightblue', # 6: Frontier
            'red'        # 7: Currently expanded node
        ])
        bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]
        norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

        # Plot the maze
        ax.imshow(viz_grid, cmap=cmap, norm=norm)
        ax.set_title(f"{self.algorithm_name} Search - Step {step_data['step']}")
        ax.set_xticks([])
        ax.set_yticks([])

        # Add legend
        legend_elements = [
            plt.Rectangle((0,0), 1, 1, color='white', label='Path'),
            plt.Rectangle((0,0), 1, 1, color='yellow', label='Visited'),
            plt.Rectangle((0,0), 1, 1, color='lightblue', label=self.get_frontier_name()),
            plt.Rectangle((0,0), 1, 1, color='red', label='Current Node'),
            plt.Rectangle((0,0), 1, 1, color='green', label='Current Path'),
            plt.Rectangle((0,0), 1, 1, color='blue', label='Start'),
            plt.Rectangle((0,0), 1, 1, color='purple', label='Goal'),
            plt.Rectangle((0,0), 1, 1, color='black', label='Wall')
        ]
        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)

    @abstractmethod
    def _plot_algorithm_state(self, ax, step_data):
        """Plot algorithm-specific state visualization."""
        pass

    @abstractmethod
    def _print_step_explanation(self, step_data):
        """Print algorithm-specific educational explanations."""
        pass

    def visualize_step(self, step_idx: int):
        """Display a single step of the algorithm with educational information."""
        if not self.steps_data or step_idx >= len(self.steps_data):
            print("Invalid step index")
            return

        step_data = self.steps_data[step_idx]
        fig = plt.figure(figsize=(18, 8))

        ax1 = plt.subplot2grid((1, 2), (0, 0))
        self._plot_maze(ax1, step_data)

        ax2 = plt.subplot2grid((1, 2), (0, 1))
        self._plot_algorithm_state(ax2, step_data)

        plt.tight_layout()
        plt.show()

        self._print_step_explanation(step_data)

    def animate_on_graph(self, output_file=None, fps=1, size=5):
        """Animate algorithm on graph representation."""
        if output_file is None:
            output_file = f"{self.algorithm_name.lower()}_graph.gif"

        try:
            G = self.create_maze_graph()
            pos = {node: (node[1], -node[0]) for node in G.nodes()}

            node_size = 300
            edge_width = 1
            path_edge_width = 3

            colors = {
                'regular': 'lightgray',
                'visited': 'yellow',
                'frontier': 'lightblue',
                'path': 'green',
                'start': 'blue',
                'end': 'purple',
                'expanded': 'red'
            }

            with tempfile.TemporaryDirectory() as temp_dir:
                frame_files = []

                # Create frames for each step
                for i, step_data in enumerate(self.steps_data):
                    plt.figure(figsize=(size, size))

                    # Draw base graph elements and all nodes
                    nx.draw_networkx_nodes(G, pos, node_color=colors['regular'], node_size=node_size)

                    # Draw visited nodes
                    visited_only = [n for n in step_data['visited']
                                if n not in step_data['frontier']
                                and (not step_data['current_path'] or n not in step_data['current_path'])
                                and n != self.env.start
                                and n != self.env.end
                                and n != step_data['expanded_node']]
                    if visited_only:
                        nx.draw_networkx_nodes(G, pos, nodelist=visited_only,
                                            node_color=colors['visited'], node_size=node_size)

                    # Draw frontier nodes - use frontier_nodes if available
                    frontier_to_plot = step_data.get('frontier_nodes', step_data['frontier'])
                    frontier_only = [n for n in frontier_to_plot
                                if (not step_data['current_path'] or n not in step_data['current_path'])
                                and n != self.env.start
                                and n != self.env.end
                                and n != step_data['expanded_node']]
                    if frontier_only:
                        nx.draw_networkx_nodes(G, pos, nodelist=frontier_only,
                                            node_color=colors['frontier'], node_size=node_size)

                    # Draw current path
                    if step_data['current_path']:
                        path_only = [n for n in step_data['current_path']
                                    if n != self.env.start
                                    and n != self.env.end
                                    and n != step_data['expanded_node']]
                        if path_only:
                            nx.draw_networkx_nodes(G, pos, nodelist=path_only,
                                                node_color=colors['path'], node_size=node_size)

                        path_edges = list(zip(step_data['current_path'][:-1], step_data['current_path'][1:]))
                        nx.draw_networkx_edges(G, pos, edgelist=path_edges,
                                            width=path_edge_width, edge_color=colors['path'])

                    # Draw special nodes
                    nx.draw_networkx_nodes(G, pos, nodelist=[step_data['expanded_node']],
                                        node_color=colors['expanded'], node_size=node_size)

                    nx.draw_networkx_nodes(G, pos, nodelist=[self.env.start],
                                        node_color=colors['start'], node_size=node_size)
                    nx.draw_networkx_nodes(G, pos, nodelist=[self.env.end],
                                        node_color=colors['end'], node_size=node_size)

                    # Draw all edges
                    nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5)

                    # Add labels
                    nx.draw_networkx_labels(G, pos, labels={node: f"{node[0]},{node[1]}" for node in G.nodes()})

                    plt.title(f"{self.algorithm_name} Graph Traversal - Step {step_data['step']} / {len(self.steps_data)}")
                    plt.axis('off')

                    frame_file = os.path.join(temp_dir, f"frame_{i:03d}.png")
                    plt.savefig(frame_file)
                    frame_files.append(frame_file)
                    plt.close()

                # Add final frame if search was successful
                if self.result.success and self.result.path:
                    self._create_final_frame(G, pos, colors, edge_width, path_edge_width, temp_dir, frame_files)

                # Create GIF
                with imageio.get_writer(output_file, mode='I', fps=fps) as writer:
                    for frame_file in frame_files:
                        image = imageio.imread(frame_file)
                        writer.append_data(image)

                print(f"Graph animation saved to {output_file}")

        except Exception as e:
            print(f"Error in {self.algorithm_name} graph animation: {e}")

    def _create_final_frame(self, G, pos, colors, edge_width, path_edge_width, temp_dir, frame_files):
        """Create final frame showing complete solution."""
        plt.figure(figsize=(6, 6))

        # Draw regular nodes
        nx.draw_networkx_nodes(G, pos, node_color=colors['regular'], node_size=300)

        # Draw visited nodes
        visited_only = [n for n in self.result.visited
                    if n not in self.result.path
                    and n != self.env.start
                    and n != self.env.end]
        if visited_only:
            nx.draw_networkx_nodes(G, pos, nodelist=visited_only,
                                node_color=colors['visited'], node_size=300)

        # Draw path nodes
        path_only = [n for n in self.result.path
                    if n != self.env.start
                    and n != self.env.end]
        if path_only:
            nx.draw_networkx_nodes(G, pos, nodelist=path_only,
                                node_color=colors['path'], node_size=300)

        # Draw start and end
        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.start],
                            node_color=colors['start'], node_size=300)
        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.end],
                            node_color=colors['end'], node_size=300)

        # Draw all edges
        nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5)

        # Highlight path edges
        if self.result.path:
            path_edges = list(zip(self.result.path[:-1], self.result.path[1:]))
            nx.draw_networkx_edges(G, pos, edgelist=path_edges,
                                width=path_edge_width, edge_color=colors['path'])

        # Add labels
        nx.draw_networkx_labels(G, pos, labels={node: f"{node[0]},{node[1]}" for node in G.nodes()})

        plt.title(f"{self.algorithm_name} Graph Traversal - Final Path ({len(self.result.path)-1 if self.result.path else 0} steps)")
        plt.axis('off')

        frame_file = os.path.join(temp_dir, "frame_final.png")
        plt.savefig(frame_file)
        frame_files.append(frame_file)
        plt.close()

    def create_gif(self, filename=None, fps=1, dpi=100):
        """Create a GIF animation from algorithm steps."""
        if filename is None:
            filename = f"{self.algorithm_name.lower()}_animation.gif"

        print(f"Creating GIF with {len(self.steps_data)} frames...")

        with tempfile.TemporaryDirectory() as temp_dir:
            frame_files = []
            # Create frames for each exploration step
            for i, step_data in enumerate(self.steps_data):
                fig = plt.figure(figsize=(18, 8))

                ax1 = plt.subplot2grid((1, 2), (0, 0))
                self._plot_maze(ax1, step_data)

                ax2 = plt.subplot2grid((1, 2), (0, 1))
                self._plot_algorithm_state(ax2, step_data)

                plt.tight_layout()

                frame_file = os.path.join(temp_dir, f"frame_{i:03d}.png")
                plt.savefig(frame_file, dpi=dpi)
                frame_files.append(frame_file)
                plt.close(fig)

            # Add final solution frame if search was successful
            if self.result.success and self.result.path:
                # Create final frame showing complete solution
                fig = plt.figure(figsize=(18, 8))

                # Left side: maze with complete solution path
                ax1 = plt.subplot2grid((1, 2), (0, 0))

                # Create a mock step_data for the final state
                final_step_data = {
                    'step': self.steps_data[-1]['step'] + 1 if self.steps_data else 1,
                    'expanded_node': self.env.end,
                    'neighbors_added': [],
                    'frontier_before': [],
                    'frontier_after': [],
                    'frontier': [],
                    'visited': self.result.visited,
                    'current_path': self.result.path,
                    'visited_count': len(self.result.visited),
                    'frontier_size': 0
                }

                self._plot_maze(ax1, final_step_data)
                ax1.set_title(f"{self.algorithm_name} Search - Final Solution")

                # Right side: solution metrics
                ax2 = plt.subplot2grid((1, 2), (0, 1))
                ax2.axis('off')

                # Create solution summary table
                table_data = [
                    ["Total Steps", str(self.result.steps)],
                    ["Path Length", str(len(self.result.path))],
                    ["Visited Nodes", str(len(self.result.visited))],
                    ["Execution Time", f"{self.result.execution_time:.3f}s"],
                    ["Efficiency", f"{len(self.result.path)/len(self.result.visited)*100:.1f}%"]
                ]

                solution_table = ax2.table(
                    cellText=table_data,
                    colLabels=["Metric", "Value"],
                    colWidths=[0.3, 0.7],
                    loc='center',
                    cellLoc='center',
                    bbox=[0.1, 0.4, 0.8, 0.5]
                )
                solution_table.auto_set_font_size(False)
                solution_table.set_fontsize(12)

                # Style table
                for i in range(len(table_data) + 1):
                    for j in range(2):
                        cell = solution_table[i, j]
                        cell.set_edgecolor('black')
                        if i == 0:  # Header
                            cell.set_facecolor('#4472C4')
                            cell.set_text_props(color='white', fontweight='bold')
                        else:
                            cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

                # Add completion message
                ax2.text(0.5, 0.8, "SEARCH COMPLETED", ha='center', va='center',
                        fontsize=18, fontweight='bold', color='green')
                ax2.text(0.5, 0.2, f"Solution path found with {len(self.result.path)-1} steps",
                        ha='center', va='center', fontsize=14)

                plt.tight_layout()

                frame_file = os.path.join(temp_dir, "frame_final.png")
                plt.savefig(frame_file, dpi=dpi)
                frame_files.append(frame_file)
                plt.close(fig)

            # Create GIF
            with imageio.get_writer(filename, mode='I', fps=fps) as writer:
                for frame_file in frame_files:
                    image = imageio.imread(frame_file)
                    writer.append_data(image)

        print(f"GIF animation saved to {filename}")


class BFSDashboard(SearchAlgorithmDashboard):
    """Educational dashboard for visualizing Breadth-First Search algorithm."""

    def get_frontier_name(self) -> str:
        return "Queue (Frontier)"

    def _extract_history_data(self):
        self.steps_data = []

        for i, state in enumerate(self.result.exploration_history):
            visited, frontier, current_path, step_info = state

            # Get queue-specific information
            frontier_before = step_info.get('queue_before', [])
            frontier_after = step_info.get('queue_after', [])

            self.steps_data.append({
                'step': step_info['step'],
                'expanded_node': step_info['expanded_node'],
                'neighbors_added': step_info['neighbors_added'],
                'frontier_before': frontier_before,
                'frontier_after': frontier_after,
                'frontier': frontier_after,
                'visited': list(visited),
                'current_path': current_path,
                'visited_count': len(visited),
                'frontier_size': len(frontier)
            })

    def _plot_algorithm_state(self, ax, step_data):
        ax.axis('off')

        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Create main metrics table
        table_data = [
            ["Step", str(step_data['step'])],
            ["Expanded Node", format_node(step_data['expanded_node'])],
            ["Visited Nodes", str(step_data['visited_count'])],
            ["Path Length", str(len(step_data['current_path']) if step_data['current_path'] else 0)]
        ]

        # Format neighbors
        neighbors_text = (", ".join([format_node(n) for n in step_data['neighbors_added']])
                          if step_data['neighbors_added'] else "None (all neighbors already visited)")
        table_data.append(["Neighbors Added", neighbors_text])

        # Draw main table
        main_table = ax.table(
            cellText=table_data,
            colLabels=["Metric", "Value"],
            colWidths=[0.3, 0.7],
            loc='center',
            cellLoc='center',
            bbox=[0.1, 0.65, 0.8, 0.3]
        )
        main_table.auto_set_font_size(False)
        main_table.set_fontsize(12)

        # Style table
        for i in range(len(table_data) + 1):
            for j in range(2):
                cell = main_table[i, j]
                cell.set_edgecolor('black')
                if i == 0:  # Header
                    cell.set_facecolor('#4472C4')
                    cell.set_text_props(color='white', fontweight='bold')
                else:
                    cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

        # Queue visualization
        self._draw_frontier_table(ax, step_data, "Queue BEFORE", "Queue AFTER")

        # Add operation explanation
        operation_explanation = [
            f"1. Dequeued {format_node(step_data['expanded_node'])} from front of queue",
            f"2. Checked if it's the goal node",
            f"3. Examined unvisited neighbors"
        ]
        if step_data['neighbors_added']:
            neighbors = ", ".join([format_node(n) for n in step_data['neighbors_added']])
            operation_explanation.append(f"4. Added neighbors to queue: {neighbors}")
        else:
            operation_explanation.append("4. No new neighbors to add")

        # Draw explanation box
        ax.text(0.5, 0.1, "\n".join(operation_explanation), ha='center', va='center',
                fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor='#FFFFCC', alpha=0.5))

    def _draw_frontier_table(self, ax, step_data, before_label, after_label):
        """Draw frontier tables (before and after expansion)."""
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Draw "BEFORE" label and table
        ax.text(0.3, 0.5, before_label, ha='center', va='center', fontsize=14, fontweight='bold')

        queue_before_data = []
        for i, node in enumerate(step_data['frontier_before']):
            queue_before_data.append([f"Queue[{i}]", format_node(node)])

        if queue_before_data:
            queue_before_table = ax.table(
                cellText=queue_before_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.05, 0.2, 0.4, 0.25]
            )
            queue_before_table.auto_set_font_size(False)
            queue_before_table.set_fontsize(12)

            # Style table
            for i in range(len(queue_before_data) + 1):
                for j in range(2):
                    cell = queue_before_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#ED7D31')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#FBE5D6' if i % 2 else '#FDF2EA')
        else:
            ax.text(0.3, 0.3, "Queue was empty", ha='center', va='center', fontsize=12, color='red')

        # Draw "AFTER" label and table
        ax.text(0.7, 0.5, after_label, ha='center', va='center', fontsize=14, fontweight='bold')

        queue_after_data = []
        for i, node in enumerate(step_data['frontier_after']):
            queue_after_data.append([f"Queue[{i}]", format_node(node)])

        if queue_after_data:
            queue_after_table = ax.table(
                cellText=queue_after_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.55, 0.2, 0.4, 0.25]
            )
            queue_after_table.auto_set_font_size(False)
            queue_after_table.set_fontsize(12)

            # Style table
            for i in range(len(queue_after_data) + 1):
                for j in range(2):
                    cell = queue_after_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#70AD47')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#E2EFDA' if i % 2 else '#EAF5E0')
        else:
            ax.text(0.7, 0.3, "Queue is now empty", ha='center', va='center', fontsize=12, color='red')

    def _print_step_explanation(self, step_data):
        def format_node(node):
            return f"({node[0]},{node[1]})"

        expanded_node = format_node(step_data['expanded_node'])
        print(f"🔍 BFS Step {step_data['step']} Explanation:")
        print(f"---------------------------")
        print(f"Currently expanding: {expanded_node}")

        print("\n1️⃣ BFS Algorithm Process:")
        print(f"   - Dequeued node {expanded_node} from the front of the queue")
        print(f"   - Checked if it's the goal (it's {'the goal! 🎉' if step_data['expanded_node'] == self.env.end else 'not the goal'})")

        if step_data['expanded_node'] != self.env.end:
            if step_data['neighbors_added']:
                neighbors_text = ", ".join([format_node(n) for n in step_data['neighbors_added']])
                print(f"\n2️⃣ Discovered {len(step_data['neighbors_added'])} unvisited neighbors: {neighbors_text}")
                print("   These neighbors were added to the back of the queue for later exploration.")
                print("   💡 This is why BFS explores nodes in order of their distance from the start.")
            else:
                print("\n2️⃣ No new neighbors were discovered (all neighbors are already visited)")

        print(f"\n3️⃣ Queue status:")
        if step_data['frontier']:
            queue_text = ", ".join([format_node(n) for n in step_data['frontier']])
            print(f"   Queue now contains: {queue_text}")
            print(f"   Next node to explore will be: {format_node(step_data['frontier'][0])}")
        else:
            print("   Queue is now empty. Search will terminate.")

        print("\n4️⃣ BFS Properties:")
        print("   - BFS always finds the shortest path in unweighted graphs")
        print("   - Time complexity: O(V + E) where V is vertices and E is edges")
        print("   - Space complexity: O(V) for the queue and visited set")

        print(f"\n📊 Current Statistics:")
        print(f"   - Visited {step_data['visited_count']} nodes so far")
        print(f"   - Queue size: {step_data['frontier_size']}")
        print(f"   - Current path length: {len(step_data['current_path']) if step_data['current_path'] else 0}")


class DFSDashboard(SearchAlgorithmDashboard):
    """Educational dashboard for visualizing Depth-First Search algorithm."""

    def get_frontier_name(self) -> str:
        return "Stack (Frontier)"

    def _extract_history_data(self):
        self.steps_data = []

        for i, state in enumerate(self.result.exploration_history):
            visited, frontier, current_path, step_info = state

            # Get stack-specific information
            frontier_before = step_info.get('stack_before', [])
            frontier_after = step_info.get('stack_after', [])

            self.steps_data.append({
                'step': step_info['step'],
                'expanded_node': step_info['expanded_node'],
                'neighbors_added': step_info['neighbors_added'],
                'frontier_before': frontier_before,
                'frontier_after': frontier_after,
                'frontier': frontier_after,
                'visited': list(visited),
                'current_path': current_path,
                'visited_count': len(visited),
                'frontier_size': len(frontier)
            })

    def _plot_algorithm_state(self, ax, step_data):
        ax.axis('off')

        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Create main metrics table (same as BFS)
        table_data = [
            ["Step", str(step_data['step'])],
            ["Expanded Node", format_node(step_data['expanded_node'])],
            ["Visited Nodes", str(step_data['visited_count'])],
            ["Path Length", str(len(step_data['current_path']) if step_data['current_path'] else 0)]
        ]

        # Format neighbors
        neighbors_text = (", ".join([format_node(n) for n in step_data['neighbors_added']])
                          if step_data['neighbors_added'] else "None (all neighbors already visited)")
        table_data.append(["Neighbors Added", neighbors_text])

        # Draw main table
        main_table = ax.table(
            cellText=table_data,
            colLabels=["Metric", "Value"],
            colWidths=[0.3, 0.7],
            loc='center',
            cellLoc='center',
            bbox=[0.1, 0.65, 0.8, 0.3]
        )
        main_table.auto_set_font_size(False)
        main_table.set_fontsize(12)

        # Style table
        for i in range(len(table_data) + 1):
            for j in range(2):
                cell = main_table[i, j]
                cell.set_edgecolor('black')
                if i == 0:  # Header
                    cell.set_facecolor('#4472C4')
                    cell.set_text_props(color='white', fontweight='bold')
                else:
                    cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

        # Stack visualization
        self._draw_frontier_table(ax, step_data, "Stack BEFORE", "Stack AFTER")

        # Add operation explanation
        operation_explanation = [
            f"1. Popped {format_node(step_data['expanded_node'])} from top of stack",
            f"2. Checked if it's the goal node",
            f"3. Examined unvisited neighbors"
        ]
        if step_data['neighbors_added']:
            neighbors = ", ".join([format_node(n) for n in step_data['neighbors_added']])
            operation_explanation.append(f"4. Pushed neighbors onto stack: {neighbors}")
        else:
            operation_explanation.append("4. No new neighbors to add")

        # Draw explanation box
        ax.text(0.5, 0.1, "\n".join(operation_explanation), ha='center', va='center',
                fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor='#FFFFCC', alpha=0.5))

    def _draw_frontier_table(self, ax, step_data, before_label, after_label):
        """Draw frontier tables (before and after expansion) for DFS."""
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Draw "BEFORE" label and table
        ax.text(0.3, 0.5, before_label, ha='center', va='center', fontsize=14, fontweight='bold')

        # For stack visualization, we show top of stack first
        stack_before_data = []
        for i, node in enumerate(reversed(step_data['frontier_before'])):
            stack_before_data.append([f"Stack[{i}]", format_node(node)])

        if stack_before_data:
            stack_before_table = ax.table(
                cellText=stack_before_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.05, 0.2, 0.4, 0.25]
            )
            stack_before_table.auto_set_font_size(False)
            stack_before_table.set_fontsize(12)

            # Style table
            for i in range(len(stack_before_data) + 1):
                for j in range(2):
                    cell = stack_before_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#ED7D31')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#FBE5D6' if i % 2 else '#FDF2EA')
        else:
            ax.text(0.3, 0.3, "Stack was empty", ha='center', va='center', fontsize=12, color='red')

        # Draw "AFTER" label and table
        ax.text(0.7, 0.5, after_label, ha='center', va='center', fontsize=14, fontweight='bold')

        stack_after_data = []
        for i, node in enumerate(reversed(step_data['frontier_after'])):
            stack_after_data.append([f"Stack[{i}]", format_node(node)])

        if stack_after_data:
            stack_after_table = ax.table(
                cellText=stack_after_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.55, 0.2, 0.4, 0.25]
            )
            stack_after_table.auto_set_font_size(False)
            stack_after_table.set_fontsize(12)

            # Style table
            for i in range(len(stack_after_data) + 1):
                for j in range(2):
                    cell = stack_after_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#70AD47')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#E2EFDA' if i % 2 else '#EAF5E0')
        else:
            ax.text(0.7, 0.3, "Stack is now empty", ha='center', va='center', fontsize=12, color='red')

    def _print_step_explanation(self, step_data):
        def format_node(node):
            return f"({node[0]},{node[1]})"

        expanded_node = format_node(step_data['expanded_node'])
        print(f"🔍 DFS Step {step_data['step']} Explanation:")
        print(f"---------------------------")
        print(f"Currently expanding: {expanded_node}")

        print("\n1️⃣ DFS Algorithm Process:")
        print(f"   - Popped node {expanded_node} from the top of the stack")
        print(f"   - Checked if it's the goal (it's {'the goal! 🎉' if step_data['expanded_node'] == self.env.end else 'not the goal'})")

        if step_data['expanded_node'] != self.env.end:
            if step_data['neighbors_added']:
                neighbors_text = ", ".join([format_node(n) for n in step_data['neighbors_added']])
                print(f"\n2️⃣ Discovered {len(step_data['neighbors_added'])} unvisited neighbors: {neighbors_text}")
                print("   These neighbors were pushed onto the stack for immediate exploration.")
                print("   💡 This is why DFS explores deeply along each branch before backtracking.")
            else:
                print("\n2️⃣ No new neighbors were discovered (all neighbors are already visited)")

        print(f"\n3️⃣ Stack status:")
        if step_data['frontier']:
            stack_text = ", ".join([format_node(n) for n in step_data['frontier']])
            print(f"   Stack now contains: {stack_text}")
            print(f"   Next node to explore will be: {format_node(step_data['frontier'][-1])}")
        else:
            print("   Stack is now empty. Search will terminate.")

        print("\n4️⃣ DFS Properties:")
        print("   - DFS explores deeply along each branch before backtracking")
        print("   - May not find the shortest path in unweighted graphs")
        print("   - Time complexity: O(V + E) where V is vertices and E is edges")
        print("   - Space complexity: O(h) where h is the maximum depth of the search tree")
        print("   - Well-suited for topological sorting, detecting cycles, and maze generation")

        print(f"\n📊 Current Statistics:")
        print(f"   - Visited {step_data['visited_count']} nodes so far")
        print(f"   - Stack size: {step_data['frontier_size']}")
        print(f"   - Current path length: {len(step_data['current_path']) if step_data['current_path'] else 0}")

In [ ]:
class GreedyBestFirstDashboard(SearchAlgorithmDashboard):
    """Educational dashboard for visualizing Greedy Best-First Search algorithm."""

    def get_frontier_name(self) -> str:
        return "Priority Queue (Frontier)"

    def _extract_history_data(self):
        self.steps_data = []

        for i, state in enumerate(self.result.exploration_history):
            closed_set, frontier, current_path, step_info = state

            # Extract just the node coordinates from frontier tuples (h_value, node)
            frontier_nodes = [node for _, node in frontier] if isinstance(frontier[0], tuple) else frontier

            self.steps_data.append({
                'step': step_info['step'],
                'expanded_node': step_info['expanded_node'],
                'expanded_node_h': step_info['expanded_node_h'],
                'neighbors_added': step_info['neighbors_added'],
                'frontier_before': step_info.get('frontier_before', []),
                'frontier_after': step_info.get('frontier_after', []),
                'frontier': frontier,  # Keep original frontier for priority queue visualization
                'frontier_nodes': frontier_nodes,  # Add extracted node coordinates
                'visited': list(closed_set),
                'current_path': current_path,
                'visited_count': len(closed_set),
                'frontier_size': len(frontier)
            })

    def _plot_algorithm_state(self, ax, step_data):
        ax.axis('off')

        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Create main metrics table with heuristic value
        table_data = [
            ["Step", str(step_data['step'])],
            ["Expanded Node", format_node(step_data['expanded_node'])],
            ["Heuristic (h)", f"{step_data['expanded_node_h']:.2f}"],
            ["Visited Nodes", str(step_data['visited_count'])],
            ["Path Length", str(len(step_data['current_path']) if step_data['current_path'] else 0)]
        ]

        # Draw main table
        main_table = ax.table(
            cellText=table_data,
            colLabels=["Metric", "Value"],
            colWidths=[0.3, 0.7],
            loc='center',
            cellLoc='center',
            bbox=[0.1, 0.65, 0.8, 0.3]
        )
        main_table.auto_set_font_size(False)
        main_table.set_fontsize(12)

        # Style table
        for i in range(len(table_data) + 1):
            for j in range(2):
                cell = main_table[i, j]
                cell.set_edgecolor('black')
                if i == 0:  # Header
                    cell.set_facecolor('#4472C4')
                    cell.set_text_props(color='white', fontweight='bold')
                else:
                    cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

        # Draw priority queue visualization
        self._draw_frontier_table(ax, step_data)

        # Add explanation
        explanation = [
            f"Greedy Best-First Search:",
            f"• Expands nodes with lowest heuristic (h) value",
            f"• Current node h = {step_data['expanded_node_h']:.2f}",
            f"• Prioritizes nodes that appear closest to goal",
            f"• Does not consider path cost to node"
        ]
        ax.text(0.5, 0.1, "\n".join(explanation), ha='center', va='center',
                fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor='#FFFFCC', alpha=0.5))

    def _draw_frontier_table(self, ax, step_data):
        """Draw priority queue table with heuristic values."""
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Draw priority queue
        ax.text(0.5, 0.5, "Priority Queue (sorted by h-value)", ha='center', va='center',
                fontsize=14, fontweight='bold')

        queue_data = []
        if isinstance(step_data['frontier'], list) and step_data['frontier']:
            for i, (h, node) in enumerate(step_data['frontier']):
                queue_data.append([f"{i+1}", format_node(node), f"{h:.2f}"])

        if queue_data:
            queue_table = ax.table(
                cellText=queue_data,
                colLabels=["Position", "Node", "h-value"],
                colWidths=[0.15, 0.25, 0.2],
                loc='center',
                cellLoc='center',
                bbox=[0.2, 0.2, 0.6, 0.25]
            )
            queue_table.auto_set_font_size(False)
            queue_table.set_fontsize(12)

            # Style table
            for i in range(len(queue_data) + 1):
                for j in range(3):
                    cell = queue_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#70AD47')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#E2EFDA' if i % 2 else '#EAF5E0')
        else:
            ax.text(0.5, 0.3, "Queue is empty", ha='center', va='center', fontsize=12, color='red')

    def _print_step_explanation(self, step_data):
        def format_node(node):
            return f"({node[0]},{node[1]})"

        expanded_node = format_node(step_data['expanded_node'])
        print(f"🔍 Greedy Best-First Search Step {step_data['step']} Explanation:")
        print(f"---------------------------")
        print(f"Currently expanding: {expanded_node} with h={step_data['expanded_node_h']:.2f}")

        print("\n1️⃣ Algorithm Process:")
        print(f"   - Selected node {expanded_node} with lowest heuristic value h={step_data['expanded_node_h']:.2f}")
        print(f"   - Checked if it's the goal (it's {'the goal! 🎉' if step_data['expanded_node'] == self.env.end else 'not the goal'})")

        if step_data['expanded_node'] != self.env.end:
            if isinstance(step_data['neighbors_added'], list) and step_data['neighbors_added']:
                neighbors_text = ", ".join([f"{format_node(n)} (h={h:.2f})" for h, n in step_data['neighbors_added']])
                print(f"\n2️⃣ Discovered neighbors with heuristic values: {neighbors_text}")
                print("   These neighbors were added to the priority queue sorted by h-value.")
            else:
                print("\n2️⃣ No new neighbors were discovered (all neighbors are already visited)")

        print("\n3️⃣ Greedy Best-First Search Properties:")
        print("   - Always expands node closest to goal according to heuristic")
        print("   - Can find solutions quickly but not guaranteed to be optimal")
        print("   - May get stuck in local minima without considering path cost")

In [ ]:

class AStarDashboard(SearchAlgorithmDashboard):
    """Educational dashboard for visualizing A* Search algorithm."""

    def get_frontier_name(self) -> str:
        return "Priority Queue (Frontier)"

    def _extract_history_data(self):
        self.steps_data = []

        for i, state in enumerate(self.result.exploration_history):
            closed_set, frontier, current_path, step_info = state

            # Extract just the node coordinates from frontier tuples (f, g, node)
            frontier_nodes = [node for _, _, node in frontier] if frontier and isinstance(frontier[0], tuple) else frontier

            self.steps_data.append({
                'step': step_info['step'],
                'expanded_node': step_info['expanded_node'],
                'expanded_node_f': step_info['expanded_node_f'],
                'expanded_node_g': step_info['expanded_node_g'],
                'expanded_node_h': step_info['expanded_node_h'],
                'neighbors_added': step_info['neighbors_added'],
                'frontier_before': step_info.get('frontier_before', []),
                'frontier_after': step_info.get('frontier_after', []),
                'frontier_nodes': frontier_nodes,
                'frontier': frontier,
                'visited': list(closed_set),
                'current_path': current_path,
                'visited_count': len(closed_set),
                'frontier_size': len(frontier)
            })

    def _plot_algorithm_state(self, ax, step_data):
        ax.axis('off')

        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Create main metrics table with f, g, h values
        table_data = [
            ["Step", str(step_data['step'])],
            ["Expanded Node", format_node(step_data['expanded_node'])],
            ["f = g + h", f"{step_data['expanded_node_f']:.2f}"],
            ["g (path cost)", f"{step_data['expanded_node_g']:.2f}"],
            ["h (heuristic)", f"{step_data['expanded_node_h']:.2f}"],
            ["Visited Nodes", str(step_data['visited_count'])],
            ["Path Length", str(len(step_data['current_path']) if step_data['current_path'] else 0)]
        ]

        # Draw main table
        main_table = ax.table(
            cellText=table_data,
            colLabels=["Metric", "Value"],
            colWidths=[0.3, 0.7],
            loc='center',
            cellLoc='center',
            bbox=[0.1, 0.7, 0.8, 0.25]
        )
        main_table.auto_set_font_size(False)
        main_table.set_fontsize(12)

        # Style table
        for i in range(len(table_data) + 1):
            for j in range(2):
                cell = main_table[i, j]
                cell.set_edgecolor('black')
                if i == 0:  # Header
                    cell.set_facecolor('#4472C4')
                    cell.set_text_props(color='white', fontweight='bold')
                else:
                    cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

        # Draw priority queue visualization
        self._draw_frontier_table(ax, step_data)

        # Add explanation
        explanation = [
            f"A* Search Algorithm:",
            f"• Expands nodes with lowest f = g + h value",
            f"• g = cost from start to node ({step_data['expanded_node_g']:.2f})",
            f"• h = estimated cost to goal ({step_data['expanded_node_h']:.2f})",
            f"• f = total estimated cost ({step_data['expanded_node_f']:.2f})",
            f"• Balances path cost and goal proximity"
        ]
        ax.text(0.5, 0.15, "\n".join(explanation), ha='center', va='center',
                fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor='#FFFFCC', alpha=0.5))

    def _draw_frontier_table(self, ax, step_data):
        """Draw priority queue table with f, g, h values."""
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Draw priority queue
        ax.text(0.5, 0.55, "Priority Queue (sorted by f-value)", ha='center', va='center',
                fontsize=14, fontweight='bold')

        queue_data = []
        if isinstance(step_data['frontier'], list) and step_data['frontier']:
            for i, (f, g, node) in enumerate(step_data['frontier']):
                h = f - g  # Calculate h from f and g
                queue_data.append([f"{i+1}", format_node(node), f"{f:.2f}", f"{g:.2f}", f"{h:.2f}"])

        if queue_data:
            queue_table = ax.table(
                cellText=queue_data,
                colLabels=["Position", "Node", "f-value", "g-value", "h-value"],
                colWidths=[0.1, 0.2, 0.15, 0.15, 0.15],
                loc='center',
                cellLoc='center',
                bbox=[0.1, 0.3, 0.8, 0.2]
            )
            queue_table.auto_set_font_size(False)
            queue_table.set_fontsize(12)

            # Style table
            for i in range(len(queue_data) + 1):
                for j in range(5):
                    cell = queue_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#70AD47')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#E2EFDA' if i % 2 else '#EAF5E0')
        else:
            ax.text(0.5, 0.4, "Queue is empty", ha='center', va='center', fontsize=12, color='red')

    def _print_step_explanation(self, step_data):
        def format_node(node):
            return f"({node[0]},{node[1]})"

        expanded_node = format_node(step_data['expanded_node'])
        print(f"🔍 A* Search Step {step_data['step']} Explanation:")
        print(f"---------------------------")
        print(f"Currently expanding: {expanded_node} with f={step_data['expanded_node_f']:.2f} (g={step_data['expanded_node_g']:.2f}, h={step_data['expanded_node_h']:.2f})")

        print("\n1️⃣ Algorithm Process:")
        print(f"   - Selected node {expanded_node} with lowest f-value")
        print(f"   - f(n) = g(n) + h(n) = {step_data['expanded_node_g']:.2f} + {step_data['expanded_node_h']:.2f} = {step_data['expanded_node_f']:.2f}")
        print(f"   - Checked if it's the goal (it's {'the goal! 🎉' if step_data['expanded_node'] == self.env.end else 'not the goal'})")

        if step_data['expanded_node'] != self.env.end:
            if isinstance(step_data['neighbors_added'], list) and step_data['neighbors_added']:
                neighbors_text = ", ".join([f"{format_node(n)} (f={f:.2f}, g={g:.2f})" for f, g, n in step_data['neighbors_added']])
                print(f"\n2️⃣ Discovered neighbors with evaluation: {neighbors_text}")
                print("   These neighbors were added to the priority queue sorted by f-value.")
            else:
                print("\n2️⃣ No new neighbors were discovered (all neighbors are already visited)")

        print("\n3️⃣ A* Search Properties:")
        print("   - Combines benefits of Dijkstra's (optimal path) and Greedy (goal-directed)")
        print("   - Guaranteed to find optimal path if heuristic is admissible (never overestimates)")
        print("   - Balances exploration of promising paths with consideration of path cost")

## Reports

In [ ]:
# Example of using the enhanced BFS visualization tools
from typing import Tuple, List, Dict, Optional, Union, Callable, Set, Any
from dataclasses import dataclass, field, asdict
from collections import deque


# Create a maze with the educational BFS implementation
config = Config(maze_size=5, show_exploration=True, visualization_delay=0.8)
env = MazeEnvironment(config)

# Create enhanced BFS instance
bfs = BreadthFirstSearch(env)

# Run the search and collect educational results
result = bfs.run()

# Option 1: Use the built-in enhanced visualization
print("Running enhanced BFS visualization...")
bfs.visualize_search(result)

# Option 2: Create an educational dashboard
dashboard = BFSDashboard(env, result)

# Show how to use the dashboard
print("\n\nCreating BFS Educational Dashboard")
print("-----------------------------------")

# Visualize a specific step
print("\nShowing step 3 as an example:")
dashboard.visualize_step(3)

# Analyze BFS performance
print("\n\nBFS Performance Analysis")
print("------------------------")
print(f"Search success: {result.success}")
print(f"Total steps: {result.steps}")
print(f"Visited nodes: {len(result.visited)}")
if result.path:
    print(f"Path length: {len(result.path)}")
    print(f"Path efficiency: {len(result.path)/len(result.visited):.2f} (path length / visited nodes)")


In [ ]:
dashboard.create_gif(fps=1.5)

In [ ]:
dashboard = BFSDashboard(env, result)
dashboard.animate_on_graph(fps=3, size=6)  # Animated view

In [ ]:

# Create a maze with the educational DFS implementation
config = Config(maze_size=5, show_exploration=True, visualization_delay=0.8)
env = MazeEnvironment(config)

# Create enhanced DFS instance
dfs = DepthFirstSearch(env)

# Run the search and collect educational results
result = dfs.run()

# Option 1: Use the built-in enhanced visualization
print("Running enhanced DFS visualization...")
dfs.visualize_search(result)

# Option 2: Create an educational dashboard
dfs_dashboard = DFSDashboard(env, result)

# Show how to use the dashboard
print("\n\nCreating DFS Dashboard")
print("-----------------------------------")

# Visualize a specific step
print("\nShowing step 3 as an example:")
dfs_dashboard.visualize_step(3)

# Analyze DFS performance
print("\n\nDFS Performance Analysis")
print("------------------------")
print(f"Search success: {result.success}")
print(f"Total steps: {result.steps}")
print(f"Visited nodes: {len(result.visited)}")
if result.path:
    print(f"Path length: {len(result.path)}")
    print(f"Path efficiency: {len(result.path)/len(result.visited):.2f} (path length / visited nodes)")

dfs_dashboard.create_gif(fps=1.5)
dfs_dashboard.animate_on_graph(fps=3, size=6)  # Animated view

In [ ]:
# Create a maze with the educational GreedyBestFirstSearch implementation
config = Config(maze_size=5, show_exploration=True, visualization_delay=0.8)
env = MazeEnvironment(config)

# Create enhanced GreedyBestFirstSearch instance
gbs = GreedyBestFirstSearch(env)

# Run the search and collect educational results
result = gbs.run()

# Option 1: Use the built-in enhanced visualization
print("Running enhanced BFS visualization...")
gbs.visualize_search(result)

# Option 2: Create an educational dashboard
gbs_dashboard = GreedyBestFirstDashboard(env, result)

# Show how to use the dashboard
print("\n\nCreating GreedyBestFirstSearch Dashboard")
print("-----------------------------------")

# Visualize a specific step
print("\nShowing step 3 as an example:")
gbs_dashboard.visualize_step(3)

# Analyze BFS performance
print("\n\nGreedyBestFirstSearch Performance Analysis")
print("------------------------")
print(f"Search success: {result.success}")
print(f"Total steps: {result.steps}")
print(f"Visited nodes: {len(result.visited)}")
if result.path:
    print(f"Path length: {len(result.path)}")
    print(f"Path efficiency: {len(result.path)/len(result.visited):.2f} (path length / visited nodes)")

gbs_dashboard.create_gif(fps=1.5)
gbs_dashboard.animate_on_graph(fps=3, size=6)  # Animated view

In [ ]:
# Create a maze with the educational BFS implementation
config = Config(maze_size=5, show_exploration=True, visualization_delay=0.8)
env = MazeEnvironment(config)

# Create enhanced BFS instance
a_star_search = AStarSearch(env)

# Run the search and collect educational results
result = a_star_search.run()

# Option 1: Use the built-in enhanced visualization
print("Running enhanced BFS visualization...")
a_star_search.visualize_search(result)

# Option 2: Create an educational dashboard
ass_dashboard = AStarDashboard(env, result)

# Show how to use the dashboard
print("\n\nCreating A* Dashboard")
print("-----------------------------------")

# Visualize a specific step
print("\nShowing step 3 as an example:")
ass_dashboard.visualize_step(3)

# Analyze BFS performance
print("\n\nA* Performance Analysis")
print("------------------------")
print(f"Search success: {result.success}")
print(f"Total steps: {result.steps}")
print(f"Visited nodes: {len(result.visited)}")
if result.path:
    print(f"Path length: {len(result.path)}")
    print(f"Path efficiency: {len(result.path)/len(result.visited):.2f} (path length / visited nodes)")

ass_dashboard.create_gif(fps=1.5)
ass_dashboard.animate_on_graph(fps=3, size=6)  # Animated view